In [2]:
%load_ext autoreload
%autoreload 2

In [46]:
def get_df_class_id_model(df):
    def split_path(row):
        path = row.name[0].strip()
        row['id'] = path
        row['class'] = path
        if(len(path) > 3):
            row['id'] = path.split(' ')[-1] 
            row['class'] = path.split(' ')[-2]
        return row
    df = df.apply(split_path, axis=1)
    df = df.reset_index()
    df = df.drop(columns='path')
    df = df.set_index(['class', 'id', 'model', 'run id'])
    return df


def get_df_aggregate(df, index, values, df_slice=None):
    if df_slice:
        df = df.loc[df_slice]
    all_index = ['class', 'id', 'run id']
    columns = 'model'
    aggfunc = ['mean']
    df = df.reset_index()
    df = df.pivot_table(index=index, values=values, aggfunc=aggfunc, columns=columns)
    for i in all_index:
        if(not i in index):
            df[i] = 'all'
    df = df.reset_index()
    df = df.set_index(['class', 'id', 'run id'])
    return df

def get_df_from_path(path):
    df = pd.read_hdf(path)
    df = get_df_class_id_model(df)
    return df



## Individual Analysis

In [47]:
path_input_individual = f"{path}results_individual"
df_individual = get_df_from_path(path_input_individual)
df_individual

fit time    roc time  roc value
class id    model         run id                                    
valve id_00 MFCCMix M     0        147.282579   41.764812   0.978321
            Baseline      1       1086.438118   41.841480   0.703481
            MFCCMix S+C   2        157.394463   39.984109   0.781371
            Baseline      3       1081.636285   41.878306   0.711037
            MFCCMix M+C   4        160.105759   39.783148   0.790834
...                                       ...         ...        ...
            MFCCMix M+S   248      152.604399   42.409173   0.714427
            MFCCMix M+S+C 249      154.802360   43.317260   0.947320
            MFCCMix S     250      154.570388   43.439360   0.836876
                          251      154.132209   38.858208   0.477509
            MFCCMix M+S+C 252      154.776499  289.728334   0.497083

[253 rows x 3 columns]

In [48]:
df_slice = (slice(None), slice(None), slice(None))

In [64]:
index = []
values = ['roc value']
df_individual_agg = get_df_aggregate(df_individual, index, values, df_slice)
df_individual_agg

index      mean                                  \
model                        Baseline MFCCMix C MFCCMix M MFCCMix M+C   
class id  run id                                                        
all   all all     roc value  0.725884  0.817553  0.662854    0.802593   

                                                                  
model            MFCCMix M+S MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class id  run id                                                  
all   all all       0.788949      0.831156  0.696336     0.81191

In [53]:
index = ['class']
values = ['roc value']
df_individual_agg = get_df_aggregate(df_individual, index, values, df_slice)
df_individual_agg


mean                                              \
                  roc value                                               
model              Baseline MFCCMix C MFCCMix M MFCCMix M+C MFCCMix M+S   
class  id  run id                                                         
fan    all all     0.807394  0.633128  0.664137    0.863653    0.810198   
pump   all all     0.813318  0.754199  0.690322    0.873762    0.855261   
slider all all     0.797661  0.949824  0.768896    0.825632    0.826718   
valve  all all     0.485163  0.933059  0.528060    0.647327    0.663621   

                                                       
                                                       
model             MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class  id  run id                                      
fan    all all         0.855882  0.573515    0.640942  
pump   all all         0.890826  0.666765    0.757863  
slider all all         0.857467  0.859927    0.948627  
valve  all all         0.720448  0.685138    0.900208

In [57]:
index = ['class', 'id']
values = ['roc value']
df_individual_agg = get_df_aggregate(df_individual, index, values, df_slice)
df_individual_agg

mean                                              \
                   roc value                                               
model               Baseline MFCCMix C MFCCMix M MFCCMix M+C MFCCMix M+S   
class  id   run id                                                         
fan    id00 all     0.616283  0.614617  0.570948    0.731094    0.697672   
       id02 all     0.867555  0.706479  0.718554    0.899602    0.851592   
       id04 all     0.760110  0.550453  0.642709    0.907110    0.832687   
       id06 all     0.985630  0.660963  0.724337    0.916805    0.858841   
pump   id00 all     0.771529  0.916752  0.764226    0.928081    0.929923   
       id02 all     0.711279  0.828261  0.739307    0.797284    0.787084   
       id04 all     0.971767  0.559333  0.731100    0.928467    0.894667   
       id06 all     0.798699  0.712450  0.526656    0.841215    0.809368   
slider id00 all     0.983328  0.993141  0.789526    0.973622    0.908829   
       id02 all     0.810209  0.953611  0.860479    0.871902    0.894088   
       id04 all     0.784939  0.991626  0.796027    0.831450    0.839940   
       id06 all     0.612170  0.860918  0.629550    0.625552    0.664015   
valve  id00 all     0.326507  0.959066  0.519996    0.644611    0.738837   
       id02 all     0.564190  0.982801  0.479259    0.627847    0.637338   
       id04 all     0.551181  0.963449  0.600880    0.718310    0.737014   
       id06 all     0.498773  0.826921  0.512106    0.598542    0.541296   

                                                        
                                                        
model              MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class  id   run id                                      
fan    id00 all         0.729533  0.534743    0.610685  
       id02 all         0.905347  0.624268    0.707751  
       id04 all         0.887637  0.530415    0.563774  
       id06 all         0.901011  0.604635    0.681558  
pump   id00 all         0.948066  0.829429    0.893801  
       id02 all         0.847036  0.602359    0.787842  
       id04 all         0.887033  0.674333    0.642167  
       id06 all         0.881167  0.560938    0.707644  
slider id00 all         0.993356  0.984879    0.990474  
       id02 all         0.923078  0.726868    0.924598  
       id04 all         0.853501  0.949480    0.971973  
       id06 all         0.659934  0.778479    0.907461  
valve  id00 all         0.792764  0.773862    0.944119  
       id02 all         0.657500  0.666181    0.933333  
       id04 all         0.800093  0.635509    0.923634  
       id06 all         0.631435  0.665000    0.799745

In [45]:
index = ['class', 'id', 'run id']
values = ['roc value']
df_individual_agg = get_df_aggregate(df_individual, index, values, df_slice)
df_individual_agg

mean              
                   roc value              
model               Baseline MFCCMix M+S+C
class  id   run id                        
fan    id00 0       0.625063      0.731402
            1       0.604537      0.721429
            2       0.619249      0.735767
       id02 0       0.868188      0.930129
            1       0.856922      0.914091
            2       0.877554      0.871820
       id04 0       0.768257      0.878741
            1       0.767051      0.887163
            2       0.745021      0.897006
       id06 0       0.986472      0.884953
            1       0.986226      0.910820
            2       0.984193      0.907260
pump   id00 0       0.766052      0.946648
            1       0.771431      0.943469
            2       0.777104      0.954081
       id02 0       0.701323      0.864784
            1       0.727863      0.836133
            2       0.704651      0.840192
       id04 0       0.981400      0.883500
            1       0.967200      0.859800
            2       0.966700      0.917800
       id06 0       0.762303      0.868608
            1       0.794214      0.880815
            2       0.839581      0.894079
slider id00 0       0.980708      0.991313
            1       0.982617      0.989742
            2       0.986657      0.999014
       id02 0       0.820940      0.921250
            1       0.790571      0.929624
            2       0.819117      0.918360
       id04 0       0.778500      0.857688
            1       0.806527      0.878172
            2       0.769789      0.824643
       id06 0       0.590330      0.680722
            1       0.658629      0.645373
            2       0.587552      0.653705
valve  id00 0       0.281336      0.840477
            1       0.338324      0.762234
            2       0.359862      0.775581
       id02 0       0.530833      0.656806
            1       0.578750      0.642569
            2       0.582986      0.673125
       id04 0       0.550278      0.808819
            1       0.565486      0.794444
            2       0.537778      0.797014
       id06 0       0.466458      0.603750
            1       0.488264      0.623611
            2       0.541597      0.666944

## Combined Analysis

In [24]:
path_input_combined = f"{path}results_combined"
df_combined = get_df_from_path(path_input_combined)
df_combined

fit time  roc time  roc value
class          id    model     run id                               
machine_type_1 id_00 MFCCMix M 0       1.071858  1.121786       1.00
               id_04 MFCCMix M 1       1.093579  1.105280       0.84
               id_00 MFCCMix M 2       2.890638  1.010204       0.68
               id_04 MFCCMix M 3       1.094739  1.116160       0.72
               id_02 MFCCMix M 4       1.094001  1.133934       0.64
               id_04 MFCCMix M 5       1.122451  1.324407       0.76
               id_02 MFCCMix M 6       1.091670  1.078403       0.88
               id_00 MFCCMix M 7       1.113179  1.147081       0.84
               id_02 MFCCMix M 8       1.154819  1.117413       0.88

In [50]:
index = []
values = ['roc value']
df_combined_agg = get_df_aggregate(df_combined, index, values, df_slice)
df_combined_agg

index      mean                                  \
model                        Baseline MFCCMix C MFCCMix M MFCCMix M+C   
class id  run id                                                        
all   all all     roc value  0.576767  0.535295  0.581274     0.59613   

                                                                  
model            MFCCMix M+S MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class id  run id                                                  
all   all all       0.585171      0.598701  0.569857    0.554155

In [56]:
index = ['class']
values = ['roc value']
df_combined_agg = get_df_aggregate(df_combined, index, values, df_slice)
df_combined_agg

mean                                              \
                  roc value                                               
model              Baseline MFCCMix C MFCCMix M MFCCMix M+C MFCCMix M+S   
class  id  run id                                                         
fan    all all     0.480623  0.547823  0.626480    0.626231    0.600869   
pump   all all     0.597953  0.623581  0.608585    0.633559    0.555636   
slider all all     0.728740  0.615290  0.599097    0.619778    0.640125   
valve  all all     0.499751  0.354487  0.490935    0.504953    0.544052   

                                                       
                                                       
model             MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class  id  run id                                      
fan    all all         0.661890  0.526168    0.536053  
pump   all all         0.596907  0.571933    0.583847  
slider all all         0.678314  0.686557    0.701620  
valve  all all         0.457692  0.494769    0.395101

In [58]:
index = ['class', 'id']
values = ['roc value']
df_combined_agg = get_df_aggregate(df_combined, index, values, df_slice)
df_combined_agg

mean                                              \
                   roc value                                               
model               Baseline MFCCMix C MFCCMix M MFCCMix M+C MFCCMix M+S   
class  id   run id                                                         
fan    id00 all     0.584497  0.573109  0.596518    0.573095    0.564000   
       id02 all     0.423729  0.627982  0.613688    0.605023    0.626012   
       id04 all     0.596963  0.543150  0.604893    0.597368    0.568448   
       id06 all     0.317301  0.447050  0.690820    0.729437    0.645018   
pump   id00 all     0.752441  0.726392  0.864019    0.843823    0.781391   
       id02 all     0.782485  0.612991  0.764115    0.767064    0.705814   
       id04 all     0.227800  0.471100  0.209800    0.239767    0.295733   
       id06 all     0.629085  0.683840  0.596405    0.683583    0.439607   
slider id00 all     0.978015  0.954375  0.617814    0.686940    0.794894   
       id02 all     0.733951  0.756966  0.741395    0.758074    0.772139   
       id04 all     0.637619  0.225098  0.598557    0.637388    0.549078   
       id06 all     0.565375  0.524723  0.438623    0.396709    0.444388   
valve  id00 all     0.475626  0.325354  0.495892    0.522962    0.577761   
       id02 all     0.563310  0.615486  0.533819    0.570602    0.598241   
       id04 all     0.498426  0.219699  0.559375    0.511181    0.536898   
       id06 all     0.461644  0.257407  0.374653    0.415069    0.463310   

                                                        
                                                        
model              MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class  id   run id                                      
fan    id00 all         0.577375  0.547127    0.564575  
       id02 all         0.720116  0.590276    0.587268  
       id04 all         0.649546  0.467948    0.506677  
       id06 all         0.700522  0.499323    0.485690  
pump   id00 all         0.831630  0.674018    0.714444  
       id02 all         0.776587  0.471039    0.493493  
       id04 all         0.144400  0.508433    0.486767  
       id06 all         0.635012  0.634243    0.640683  
slider id00 all         0.822105  0.984477    0.980077  
       id02 all         0.792521  0.753249    0.727952  
       id04 all         0.618220  0.504903    0.433594  
       id06 all         0.480411  0.503598    0.664857  
valve  id00 all         0.477156  0.631947    0.328390  
       id02 all         0.564815  0.624120    0.571690  
       id04 all         0.444398  0.377222    0.417639  
       id06 all         0.344398  0.345787    0.262685

In [12]:
index = ['class', 'id', 'run id']
df_combined_agg = get_df_aggregate(df_combined, index)
df_combined_agg

mean                                              \
model               Baseline MFCCMix C MFCCMix M MFCCMix M+C MFCCMix M+S   
class  id   run id                                                         
fan    id00 0       0.589753  0.572602  0.580963    0.574480    0.558621   
            1       0.584290  0.582110  0.612053    0.563475    0.548775   
            2       0.579448  0.564616  0.596538    0.581332    0.584604   
       id02 0       0.578068  0.631955  0.615645    0.550128    0.608996   
            1       0.329133  0.617314  0.625585    0.649576    0.683328   
            2       0.363987  0.634679  0.599832    0.615366    0.585711   
       id04 0       0.589130  0.558693  0.510313    0.598106    0.615900   
            1       0.584605  0.536118  0.657014    0.610830    0.539784   
            2       0.617156  0.534640  0.647353    0.583168    0.549660   
       id06 0       0.319595  0.425588  0.680121    0.716677    0.649243   
            1       0.305177  0.446697  0.757568    0.788538    0.572755   
            2       0.327131  0.468865  0.634771    0.683098    0.713055   
pump   id00 0       0.756663  0.729473  0.845127    0.846985    0.821654   
            1       0.753533  0.730500  0.886303    0.856521    0.759157   
            2       0.747127  0.719204  0.860629    0.827962    0.763363   
       id02 0       0.769337  0.542975  0.767146    0.744420    0.694668   
            1       0.786056  0.667965  0.704164    0.798231    0.712036   
            2       0.792062  0.628033  0.821037    0.758542    0.710738   
       id04 0       0.328800  0.486500  0.259900    0.193500    0.258700   
            1       0.222700  0.471600  0.209800    0.192000    0.285500   
            2       0.131900  0.455200  0.159700    0.333800    0.343000   
       id06 0       0.603422  0.680219  0.577470    0.673683    0.261246   
            1       0.593522  0.689158  0.557958    0.700404    0.459439   
            2       0.690311  0.682141  0.653787    0.676663    0.598135   
slider id00 0       0.979177  0.969961  0.524807    0.695177    0.712016   
            1       0.971168  0.961526  0.608667    0.649097    0.877896   
            2       0.983698  0.931637  0.719969    0.716545    0.794770   
       id02 0       0.736509  0.756133  0.646453    0.746006    0.837072   
            1       0.726311  0.740577  0.824405    0.762740    0.828809   
            2       0.739034  0.774187  0.753328    0.765476    0.650535   
       id04 0       0.646983  0.283077  0.584143    0.638714    0.610466   
            1       0.626720  0.190759  0.722952    0.660744    0.444925   
            2       0.639155  0.201458  0.488575    0.612707    0.591844   
       id06 0       0.538568  0.490595  0.451206    0.362707    0.397298   
            1       0.546522  0.523545  0.406262    0.395026    0.385431   
            2       0.611034  0.560030  0.458402    0.432395    0.550436   
valve  id00 0       0.327449  0.431961  0.507097    0.529765    0.438387   
            1       0.551868  0.310501  0.497140    0.529694    0.730316   
            2       0.547560  0.233599  0.483440    0.509427    0.564579   
       id02 0       0.589444  0.673889  0.540208    0.565278    0.590972   
            1       0.529306  0.582569  0.555347    0.576736    0.589028   
            2       0.571181  0.590000  0.505903    0.569792    0.614722   
       id04 0       0.511667  0.277222  0.531667    0.521667    0.578819   
            1       0.498472  0.136528  0.561806    0.554653    0.560764   
            2       0.485139  0.245347  0.584653    0.457222    0.471111   
       id06 0       0.394028  0.257083  0.425625    0.434167    0.506042   
            1       0.518611  0.293542  0.341736    0.411597    0.447778   
            2       0.472292  0.221597  0.356597    0.399444    0.436111   

                                                        
model              MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class  id   run id               